In [12]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(".."))  # go up from notebooks/ to the project root

from data.df_cleaning import df_cleaned_result

# load your CSV here, manually:
df = pd.read_csv('../data/Bird_Migration_Data_with_Origin.csv')

print(df_cleaned_result.head())

Initial DataFrame loaded with 10000 records.

--- Initial Data Head ---
  Bird_ID  Species         Region    Habitat Weather_Condition  \
0   B1000  Warbler  South America  Grassland            Stormy   
1   B1001    Stork  North America  Grassland            Stormy   
2   B1002     Hawk  South America   Mountain            Stormy   
3   B1003  Warbler  South America      Urban            Stormy   
4   B1004    Crane         Europe      Urban             Windy   

  Migration_Reason  Start_Latitude  Start_Longitude  End_Latitude  \
0          Feeding       11.906566      -169.378251     30.377647   
1         Breeding       62.301546      -111.475069     39.921092   
2  Avoid Predators       87.861164       -78.727327     66.990980   
3   Climate Change       35.770590       153.104341    -49.003145   
4  Avoid Predators      -21.611614       106.674824     11.681051   

   End_Longitude  ...  Migration_Start_Month  Migration_End_Month  Rest_Stops  \
0     -21.366879  ...              